In [ ]:
import pandas as pd

: 

In [14]:
# Define file pathname
base = "/Users/ziadharmanani/Documents/GitHub/INF1340-Ridership/Ridership-Data/Bike share ridership "
extension = [
    "2023-01", "2023-02", "2023-03", "2023-04", "2023-05", "2023-06",
    "2023-07", "2023-08", "2023-09", "2023-10", "2023-11", "2023-12",
    ]

# Define columns order in a list
columns = [
    "Trip Id", "Trip  Duration", "Start Station Id", "Start Time",
    "Start Station Name", "End Station Id", "End Time",
    "End Station Name", "Bike Id", "User Type"
    ]

# Used to store file paths
paths = []

# Loop through file extensions and combine them to locate them individually
for month in extension:
    path = base + month + ".csv"
    paths.append(path)

# Used to store the individual data frames
all_dfs = []

# Clean columns (Inconsistencies between 2024 and other years)
# Mismatch beteen the years issued a lot of NaN values. Trip Id was duplicated (2024 had its own).
for file in paths:
    df_temp = pd.read_csv(file, encoding = "latin1")
    df_temp.columns = [col.replace("ï»¿", "").strip() for col in df_temp.columns]

    # Used to clean BOM character for "Trip Id" (2024)
    if "ï»¿Trip Id" in df_temp.columns:
        df_temp["Trip Id"] = df_temp["ï»¿Trip Id"]
        df_temp = df_temp.drop(columns = ["ï»¿Trip Id"])
    
    # Reindexes the data frame according to columns list.
    df_temp = df_temp.reindex(columns = columns)
    all_dfs.append(df_temp)

# Concatinate dataframe
df = pd.concat(all_dfs, ignore_index = True)

# Drop "Model" to stay consistent with previous years (Only present in 2024)
df = df.drop(columns = ["Model", "Bike Id"], errors = "ignore")

#remove extra space in Trip Duration column
#df.columns = df.columns.str.replace("Trip  Duration", "Trip Duration")

# NEXT STEPS: Look into Start Station Name and End Station Name.


In [15]:
df.sample(10)

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,User Type
2059410,22519311,4346,7549,06/24/2023 12:51,439 Sherbourne St,7317.0,06/24/2023 14:03,Hubbard Blvd / Balsam Av,Casual Member
4642793,25470898,1742,7246,10/09/2023 19:23,Yonge St / Bloor St,7006.0,10/09/2023 19:52,Bay St / College St (East Side),Casual Member
4741104,25583383,297,7066,10/14/2023 18:56,Willcocks St / St. George St,7248.0,10/14/2023 19:01,Baldwin Ave / Spadina Ave - SMART,Casual Member
748800,21006766,776,7032,04/14/2023 18:59,Augusta Ave / Dundas St W,7193.0,04/14/2023 19:12,Queen St W / Gladstone Ave,Casual Member
5685213,26651514,1383,7716,12/26/2023 14:24,NaN,7013.0,12/26/2023 14:47,Scott St / The Esplanade,Casual Member
1674028,22079024,1683,7260,06/06/2023 16:36,Spadina Ave / Adelaide St W,7314.0,06/06/2023 17:04,Queen St. E / Eastern Ave,Casual Member
1871351,22306424,609,7534,06/16/2023 16:49,Walnut Ave / Queen St W,7534.0,06/16/2023 17:00,Walnut Ave / Queen St W,Casual Member
1187517,21517575,365,7419,05/14/2023 14:51,Bloor St W / Huron St,7078.0,05/14/2023 14:57,College St / Major St,Annual Member
5562542,26515335,668,7053,12/11/2023 16:10,Metro Hall Plaza,7046.0,12/11/2023 16:22,Niagara St / Richmond St W,Casual Member
3783174,24493442,677,7031,09/04/2023 09:57,Jarvis St / Isabella St,7063.0,09/04/2023 10:08,Queen's Park / Bloor St W,Casual Member


In [24]:
print(f"Start Station Name: {df['Start Station Name'].nunique()}")
print(f"End Station Name: {df['End Station Name'].nunique()}")

Start Station Name: 593
End Station Name: 592


In [17]:
df.isna().sum()

Trip Id                    0
Trip  Duration             0
Start Station Id           0
Start Time                 0
Start Station Name    595075
End Station Id          2944
End Time                   0
End Station Name      598563
User Type                  0
dtype: int64

In [ ]:
#split start time into 2 columns
#df[['Start Date', 'Start Time']] = df['Start Time'].str.split(' ', expand=True)

#move start date to the 3rd index
col = df.pop('Start Date')
df.insert(3, col.name, col)

#convert date to datetime format
df['Start Date'] = pd.to_datetime(df['Start Date'], format = '%m/%d/%Y')

In [ ]:
#split date into day/month but keep og column
df['Start Day'] = df['Start Date'].dt.day

df['Start Month'] = df['Start Date'].dt.month

#move start day to the 4th index
col = df.pop('Start Day')
df.insert(4, col.name, col)

#move start month to the 5th index
col = df.pop('Start Month')
df.insert(5, col.name, col)


#change trip duration to minutes/hours
def convert(seconds):
    seconds = seconds % (24 * 60 * 60)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60

    return "%d:%02d:%02d" % (hour, minutes, seconds)


#df['Trip Duration'] = df['Trip Duration'].apply(convert)
df.tail()

In [ ]:
#create weekday/weekend column
df['Weekday/Weekend'] = df['Start Date'].dt.weekday.apply(lambda x: 'Weekday' if x <= 5 else 'Weekend')

df.head()